# This Notebook Runs Baselines (BPOSD, etc.)

In [1]:
import json
from functools import partial
from io import StringIO
from operator import itemgetter

from surface_sim.setups.setup import SetupDict

from mccd.random_clifford_circuit import *
from surface_sim.setups import CircuitNoiseSetup
from surface_sim.models import CircuitNoiseModel, BiasedCircuitNoiseModel
from surface_sim import Detectors, Setup
from surface_sim.experiments import schedule_from_circuit, experiment_from_schedule
import time
import stim

from pathlib import Path
import stim
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
from joblib import Parallel, delayed
import itertools
import shelve
from surface_sim.layouts import rot_surface_codes

from pymatching import Matching as MWPM
from mle_decoder import MLEDecoder as MLE
from stimbposd import BPOSD
from sklearn.metrics import accuracy_score

import os

os.environ['GRB_LICENSE_FILE'] = '/Users/fengcong/.gurobi/gurobi.lic'

In [2]:
!gurobi_cl --license

Set parameter Username
Set parameter LicenseID to value 2739212
Set parameter LogFile to value "gurobi.log"
Using license file /Users/fengcong/.gurobi/gurobi.lic
Academic license - for non-commercial use only - expires 2026-11-15


## Baseline Decoders

- MWPM. We use the open-source library PyMatching with the noise model used for data generation as detailed in the ‘Experimentally motivated noise model’ subsection.

- BP-OSD. We use the open-source library stimbposd. We use the exact noise model used for data generation and set the maximal belief propagation iterations to 20.

- MLE. We use the algorithm developed and implemented as in ref. 14.

### Notes

All baselines have PyPI packages.

```
pymatching
mle-decoder
stimbposd
```

In [3]:
DECODER_BASELINES = {
    'BPOSD': partial(BPOSD, max_bp_iters=20),
    'MLE': MLE, # TODO: model too large. acacdemic license.
    'MWPM': MWPM,
}

## Basic Gates & Surface Code

MCCD uses I, X, Y, Z, H (single qubit gates) and CX (two qubit gates).

MCCD uses Rotated Surface Code.

surface-sim supports I, X, Z for rotated gates and I, H, X, Z for unrotated gates.

### Notes

We use the gates which `surface-sim` supports.

In [4]:
from surface_sim.circuit_blocks.rot_surface_code_css import gate_to_iterator
print('Rotated', gate_to_iterator.keys())
from surface_sim.circuit_blocks.unrot_surface_code_css import gate_to_iterator
print('Unrotated', gate_to_iterator.keys())
ROT_GATES = list('IXZ')
UNR_GATES = list('IHXZ')
MCCD_GATES = ['I', 'X', 'Y', 'Z', 'H']

Rotated dict_keys(['TICK', 'I', 'S', 'X', 'Z', 'CX', 'CNOT', 'R', 'RZ', 'RX', 'M', 'MZ', 'MX'])
Unrotated dict_keys(['TICK', 'I', 'S', 'H', 'X', 'Z', 'CX', 'CNOT', 'R', 'RZ', 'RX', 'M', 'MZ', 'MX'])


In [5]:
from gensim.utils import grouper
from stim import CircuitInstruction, Circuit
from surface_sim.setups import SQ_GATES, TQ_GATES, SQ_MEASUREMENTS, SQ_RESETS
from typing import override, Collection
from surface_sim import Model


class ExperimentalCircuitNoiseModel(Model):
    @override
    def __getattribute__(self, name: str) -> object:
        attr = super().__getattribute__(name)

        if not callable(attr):
            return attr

        if name in SQ_GATES:

            def sq_gate(qubits: Collection[str]) -> Circuit:
                inds = self.get_inds(qubits)
                circ = Circuit()
                circ.append(CircuitInstruction(SQ_GATES[name], inds))
                circ.append(CircuitInstruction("PAULI_CHANNEL_1", inds, [0.0001, 0.0001, 0.0001]))
                return circ

            return sq_gate

        elif name in TQ_GATES:

            def tq_gate(qubits: Collection[str]) -> Circuit:
                if len(qubits) % 2 != 0:
                    raise ValueError("Expected and even number of qubits.")

                inds = self.get_inds(qubits)
                circ = Circuit()
                circ.append(CircuitInstruction(TQ_GATES[name], inds))
                probs = [0.0005, 0.00175, 0.000625, 0.0005, 0, 0, 0, 0.00175, 0, 0, 0, 0.000625, 0, 0, 0.00125]
                circ.append(CircuitInstruction("PAULI_CHANNEL_2", inds, probs))
                return circ

            return tq_gate

        elif name in SQ_MEASUREMENTS:

            def sq_meas(qubits: Collection[str]) -> Circuit:
                inds = self.get_inds(qubits)
                noise_name = "X_ERROR" if "_x" not in name else "Z_ERROR"
                circ = Circuit()

                # separates X_ERROR and MZ lines for clearer stim.Circuits and diagrams
                prob: float = 0.002
                circ.append(CircuitInstruction(noise_name, inds, [prob]))
                for qubit in qubits:
                    self.add_meas(qubit)

                return circ

            return sq_meas

        elif name in SQ_RESETS:

            def sq_reset(qubits: Collection[str]) -> Circuit:
                inds = self.get_inds(qubits)
                noise_name = "X_ERROR" if "_x" not in name else "Z_ERROR"
                circ = Circuit()
                circ.append(CircuitInstruction(SQ_RESETS[name], inds))
                prob: float = 0.002
                circ.append(CircuitInstruction(noise_name, inds, [prob]))
                return circ

            return sq_reset

        return attr

    @override
    def idle(self, qubits: Collection[str]) -> Circuit:
        inds = self.get_inds(qubits)
        circ = Circuit()

        circ.append(CircuitInstruction("I", inds))
        circ += self.idle_noise(qubits)

        return circ

    @override
    def idle_noise(
        self, qubits: Collection[str], param_name: str = "idle_error_prob"
    ) -> Circuit:
        # inds = self.get_inds(qubits)
        circ = Circuit()
        # TODO
        return circ

    @override
    def incoming_noise(self, qubits: Collection[str]) -> Circuit:
        return Circuit()


In [ ]:
def print_random_circuit(c: RandomCliffordCircuit):
    return list(c)

def dict_product(input_dict):
    keys = input_dict.keys()
    value_lists = input_dict.values()

    # 使用itertools.product生成所有值的组合
    value_combinations = itertools.product(*value_lists)

    # 将每个值的组合与键配对，生成字典列表
    for combo in value_combinations:
        yield dict(zip(keys, combo))

def run_decoder(name: str, circuit: stim.Circuit, shots: int):
    """Runs decoder on the given circuit

    Args:
        name: decoder name
        circuit: circuit to run
        shots: number of shots

    Returns:
        A dict containing the decoder metrics.
    """
    method = DECODER_BASELINES[name](circuit.detector_error_model())
    sampler = circuit.compile_detector_sampler()
    syndrome, labels = sampler.sample(shots=shots, separate_observables=True)
    begin = time.time_ns()
    predictions = method.decode_batch(syndrome)
    end = time.time_ns()
    logical_accuracy = accuracy_score(labels, predictions)
    walltime_seconds = (end - begin) / 1e9
    return dict(
        decoder=name,
        logical_accuracy=logical_accuracy,
        walltime_seconds=walltime_seconds,
    )

def run_decoder_tasks(bench_circuits, bench_decoders, df_name):
    """Run all the baseline decoders on the benchmark circuit.

    Args:
        bench_circuits: Benchmark circuits.
        df_name: Name of the dataframe file.

    Returns:
        The result dataframe.
    """
    def run_decoder_plus(config, cir_str, **kwargs):
        res = config.copy()
        res.update(kwargs)

        kwargs['circuit'] = stim.Circuit.from_file(StringIO(cir_str))
        try:
            res.update(run_decoder(**kwargs))
        except Exception as e:
            print(e)
        else:
            print(res['decoder'], 'depth', res['depth'], 'distance', res['distance'])

        return res

    def tasks():
        for phy_cir, _, config in bench_circuits:
            for decoder in bench_decoders:
                for t in range(trial):
                    yield delayed(run_decoder_plus)(config, phy_cir,
                                                    name=decoder, shots=num_shots)

    bench_result = Parallel(n_jobs=-1, verbose=1)(tasks())

    df = pd.DataFrame.from_records(bench_result)
    df = pd.melt(df, id_vars=['decoder', 'distance', 'depth', 'circuit_type_index'],
             value_vars=['walltime_seconds', 'logical_accuracy'],
             var_name='metric',
             value_name='value')

    filename = Path(f'./data/bench/') / noise_model.__name__ / f'{df_name}.csv'
    filename.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(filename, index=False)
    print('done')
    return df

def load_circuits(root_dir: Path):
    circuits = []
    for cir_path in sorted(root_dir.glob('*_phy.stim')):
        config_path = cir_path.with_name(cir_path.stem.replace('_phy', '_config') + '.json')
        log_cir_path = cir_path.with_name(cir_path.stem.replace('_phy', '_log') + '.stim')

        phy_cir = stim.Circuit.from_file(cir_path)
        log_cir = stim.Circuit.from_file(log_cir_path)
        config = json.loads(config_path.read_text())

        circuits.append((str(phy_cir), str(log_cir), config))
    return circuits
    

## Circuit Depths

From released source_data.zip, Type I circuits have depths:

array([ 2,  4,  6,  8, 10, 12, 14, 16, 18])

Type II circuits have depths:

array([ 4,  8, 12, 16, 20, 24, 28, 32, 36])


## Noise Model

1. The experimentally motivated noise model, which is the one used in all numerical results in the main text. See Methods section for details.
2. A simplified noise model that replaces all Pauli noise channels, both single-qubit and two-qubit, in the experiment-motivated noise model with a depolarizing noise model using an average noise strength. The probabilities of idling error, reset error, and measurement error remain the same. We refer to this as the average depolarizing noise model.
3. A stronger depolarizing noise model in which the strengths of all noise channels, including single- and two-qubit depolarization, idling error, reset error, and measurement error probability, are doubled compared to the average depolarizing noise model. We refer to this as the double depolarizing noise model.

### Note

1 can only be implemented with `stim`'s builtin instructions like `PAULI_CHANNEL_2` to fill all the parameters.
Or as a custom `Model` of `surface-sim`.

This noise model is used in Results (Fig. 4 & Fig. 5).

2 & 3 can be implemented with `surface-sim`'s `CircuitNoiseModel` with a single `prob` parameter.

The paper does not release the value of the average noise probability. We set it to 1e-3.

### Details of experimentally motivated noise model.

For the numerical studies presented in the ‘Results’ section, we use the stim package for simulation. We consider a circuit-level noise model motivated by the current experimental capability of neutral atom array-based quantum computers. Specifically, we use a circuit-level noise model that includes the following physical noises:

• Each two-qubit physical gate is followed by a two-qubit Pauli noise channel with probability [0.0005, 0.00175, 0.000625, 0.0005, 0, 0, 0, 0.00175, 0, 0, 0, 0.000625, 0, 0, 0.00125]

• Each single-qubit physical gate is followed by a single-qubit depolarizing model with probability [0.0001, 0.0001, 0.0001]

• On a physical level, the atoms are moved to achieve flexible connectivity between different physical qubits. This comes at the cost of having idling error due to the extra time taken during the physical qubit movement, which is captured as a Pauli noise channel with probability [4 × 10−7, 4 × 10−7, 1.6 × 10−6]. This error channel is applied when physical qubit movement happens.

• Resetting a physical qubit has a bit flip error probability of P = 0.002.

• Measuring a physical qubit has a bit flip error probability of P = 0.002.



### Shots & Repeat
We evaluate each decoder over 20 independent runs. In each run, we randomly sample 1,000 syndrome trajectories from Type I/II circuits and average them to obtain a run-level performance estimate. We report the mean across the 20 runs, with error bars showing s.e.m.


In [7]:
num_shots = 1000
trial = 20
noise_model = average_depolarizing_noise

def get_figure4_settings():
    distance = [3, 5]
    depth = [2,  4,  6,  8, 10, 12, 14, 16, 18]
    # Type I one qubit. Type II two qubits.
    circuit_index = ['3']

    return dict_product(dict(
        distance=distance,
        depth=depth,
        circuit_index=circuit_index,
        noise_model=[noise_model],
    ))

def get_figure5_settings():
    distance = [3, 5]
    depth = [ 4,  8, 12, 16, 20, 24, 28, 32, 36]
    # Type I one qubit. Type II two qubits.
    circuit_index = ['4']

    return list(dict_product(dict(
        distance=distance,
        depth=depth,
        circuit_index=circuit_index,
        noise_model=[noise_model],
    )))

figure4_settings = get_figure4_settings()
figure5_settings = get_figure5_settings()

Load circuits.

In [ ]:
fig4_circuits = load_circuits(Path('./data/bench') / noise_model.__name__ / 'fig4/circuits')
fig5_circuits = load_circuits(Path('./data/bench') / noise_model.__name__ / 'fig5/circuits')

In [17]:
df4 = run_decoder_tasks(fig4_circuits, DECODER_BASELINES.keys(), 'fig4')


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.


BPOSD depth 10 distance 3
BPOSD depth 10 distance 3
BPOSD depth 10 distance 3
BPOSD depth 10 distance 3
BPOSD depth 10 distance 3
BPOSD depth 10 distance 3
BPOSD depth 10 distance 3
BPOSD depth 10 distance 3
BPOSD depth 10 distance 3
BPOSD depth 10 distance 3
BPOSD depth 10 distance 3
Set parameter Username
Set parameter LicenseID to value 2739212
Academic license - for non-commercial use only - expires 2026-11-15
BPOSD depth 10 distance 3
BPOSD depth 10 distance 3
Set parameter Username
Set parameter LicenseID to value 2739212
Academic license - for non-commercial use only - expires 2026-11-15
Set parameter Username
Set parameter LicenseID to value 2739212
Academic license - for non-commercial use only - expires 2026-11-15
BPOSD depth 10 distance 3
Set parameter Username
Set parameter LicenseID to value 2739212
Academic license - for non-commercial use only - expires 2026-11-15
BPOSD depth 10 distance 3
Set parameter Username
Set parameter LicenseID to value 2739212
Academic license -

[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:  1.2min


MLE depth 10 distance 3
MWPM depth 10 distance 3
MWPM depth 10 distance 3
MWPM depth 10 distance 3
MWPM depth 10 distance 3
MWPM depth 10 distance 3
MWPM depth 10 distance 3
MWPM depth 10 distance 3
MWPM depth 10 distance 3
MWPM depth 10 distance 3
MWPM depth 10 distance 3
MWPM depth 10 distance 3
MWPM depth 10 distance 3
MWPM depth 10 distance 3
MLE depth 10 distance 3
MWPM depth 10 distance 3
MWPM depth 10 distance 3
MWPM depth 10 distance 3
MWPM depth 10 distance 3
MWPM depth 10 distance 3
MWPM depth 10 distance 3
MWPM depth 10 distance 3
MLE depth 10 distance 3
MLE depth 10 distance 3
MLE depth 10 distance 3
MLE depth 10 distance 3
MLE depth 10 distance 3
MLE depth 10 distance 3
MLE depth 10 distance 3
MLE depth 10 distance 3
BPOSD depth 12 distance 3
BPOSD depth 12 distance 3
BPOSD depth 12 distance 3
BPOSD depth 12 distance 3
BPOSD depth 12 distance 3
BPOSD depth 12 distance 3
BPOSD depth 12 distance 3
BPOSD depth 12 distance 3
BPOSD depth 12 distance 3
BPOSD depth 12 distance 3


[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed:  9.7min


BPOSD depth 16 distance 3
BPOSD depth 16 distance 3
BPOSD depth 16 distance 3
BPOSD depth 16 distance 3
BPOSD depth 16 distance 3
BPOSD depth 16 distance 3
BPOSD depth 16 distance 3
BPOSD depth 16 distance 3
BPOSD depth 16 distance 3
BPOSD depth 16 distance 3
BPOSD depth 16 distance 3
BPOSD depth 16 distance 3
BPOSD depth 16 distance 3
BPOSD depth 16 distance 3
BPOSD depth 16 distance 3
BPOSD depth 16 distance 3
BPOSD depth 16 distance 3
BPOSD depth 16 distance 3
BPOSD depth 16 distance 3
BPOSD depth 16 distance 3
MLE depth 16 distance 3
MLE depth 16 distance 3
MLE depth 16 distance 3
MLE depth 16 distance 3
MLE depth 16 distance 3
MLE depth 16 distance 3
MLE depth 16 distance 3
MLE depth 16 distance 3
MLE depth 16 distance 3
MLE depth 16 distance 3
MLE depth 16 distance 3
MWPM depth 16 distance 3
MWPM depth 16 distance 3
MWPM depth 16 distance 3
MWPM depth 16 distance 3
MLE depth 16 distance 3
MWPM depth 16 distance 3
MWPM depth 16 distance 3
MWPM depth 16 distance 3
MWPM depth 16 dis

[Parallel(n_jobs=-1)]: Done 430 tasks      | elapsed: 37.3min


BPOSD depth 6 distance 3
BPOSD depth 6 distance 3
BPOSD depth 6 distance 3
BPOSD depth 6 distance 3
BPOSD depth 6 distance 3
MLE depth 6 distance 3
MLE depth 6 distance 3
MLE depth 6 distance 3
MLE depth 6 distance 3
MLE depth 6 distance 3
MLE depth 6 distance 3
MLE depth 6 distance 3
MLE depth 6 distance 3
MLE depth 6 distance 3
MLE depth 6 distance 3
MLE depth 6 distance 3
MWPM depth 6 distance 3
MWPM depth 6 distance 3
MWPM depth 6 distance 3
MWPM depth 6 distance 3
MWPM depth 6 distance 3
MWPM depth 6 distance 3
MWPM depth 6 distance 3
MWPM depth 6 distance 3
MWPM depth 6 distance 3
MWPM depth 6 distance 3
MWPM depth 6 distance 3
MWPM depth 6 distance 3
MWPM depth 6 distance 3
MWPM depth 6 distance 3
MWPM depth 6 distance 3
MWPM depth 6 distance 3
MWPM depth 6 distance 3
MWPM depth 6 distance 3
MWPM depth 6 distance 3
MWPM depth 6 distance 3
MLE depth 6 distance 3
MLE depth 6 distance 3
MLE depth 6 distance 3
MLE depth 6 distance 3
MLE depth 6 distance 3
MLE depth 6 distance 3
MLE 

KeyboardInterrupt: 

In [ ]:
df5 = run_decoder_tasks(fig5_circuits, DECODER_BASELINES.keys(), 'fig5')
